In [4]:
# scraper for news articles to aggregate banned items and highlights
import os.path
# data manipulation
import pandas as pd 
import re
# email packages
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

# creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# service = build("gmail", "v1", credentials=creds)
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

creds = None
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        try:
            creds.refresh(Request())
        except Exception:
            creds = None  # refresh token revoked -> re-auth
    if not creds:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
        token.write(creds.to_json())

service = build("gmail", "v1", credentials=creds)
#################################################################################################
# web scraping
from urllib.parse import unquote, urlparse, parse_qs
from bs4 import BeautifulSoup
import requests
# key mappings, ban words, and stop words
from ban_config import state_patterns, keywords
# NLP
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# graphs
import plotly.express as px
import us

import base64

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\makri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# initialize connection to server
all_messages = []
page_token = None

while True:
    response = service.users().messages().list(
        userId="me",
        q='subject:"Google Alert - ban"',
        maxResults=500,
        pageToken=page_token
    ).execute()

    all_messages.extend(response.get("messages", []))
    page_token = response.get("nextPageToken")

    if not page_token:
        break

print(f"Total messages found: {len(all_messages)}")

Total messages found: 2062


In [5]:
# -----------------------
# Pre-compile patterns
# -----------------------
results_pattern = re.compile(
    r"(\d+\s+new results for \[Bans\])",
    re.IGNORECASE
)

split_pattern = re.compile(
    r"(.*?)(?:\r\n\r\n|$)",
    re.DOTALL
)

line_pattern = re.compile(
    r"(.*?)(?:\r\n|$)",
    re.DOTALL
)

# -----------------------
# Helper: fetch + decode Gmail message
# -----------------------
def get_plain_text_and_date(service, msg_id):
    msg = service.users().messages().get(
        userId="me",
        id=msg_id,
        format="full"
    ).execute()

    payload = msg.get("payload", {})
    headers = payload.get("headers", [])
    parts = payload.get("parts", [])

    # Extract Date header
    alert_date = None
    for h in headers:
        if h["name"].lower() == "date":
            alert_date = h["value"]
            break

    # Extract plain text body
    for part in parts:
        if part.get("mimeType") == "text/plain":
            data = part["body"].get("data")
            if data:
                return (
                    alert_date,
                    base64.urlsafe_b64decode(data).decode(
                        "utf-8", errors="ignore"
                    )
                )

    return alert_date, None


# -----------------------
# Iterate through Gmail messages
# -----------------------
container = {
    "alert_date": [],
    "headlines": [],
    "article_links": [],
}

for m in all_messages:
    alert_date, plain_text = get_plain_text_and_date(service, m["id"])
    if not plain_text:
        continue

    # Optional debug: how many results Google reports
    match = results_pattern.search(plain_text)
    if match:
        print("Found:", match.group(1))

    # Split alert into blocks
    matches = [
        g.group(1).strip()
        for g in split_pattern.finditer(plain_text)
        if g.group(1).strip()
    ]

    # Guard against malformed alerts
    if len(matches) < 5:
        continue

    matches = matches[1:-3]

    for item in matches:
        sub_groups = [
            g.group(1).strip()
            for g in line_pattern.finditer(item)
            if g.group(1).strip()
        ]

        if not sub_groups:
            continue

        container["alert_date"].append(alert_date)
        container["headlines"].append(sub_groups[0])

        url_match = re.search(r"<(https.*?)>", sub_groups[-1])
        if url_match:
            google_url = unquote(url_match.group(1))
            parsed = urlparse(google_url)
            qs = parse_qs(parsed.query)
            container["article_links"].append(
                qs.get("url", [""])[0] or "no url found (1)."
            )
        else:
            container["article_links"].append("no url found (2).")

In [6]:
# extract states / locations
def extract_state(text: str) -> str:
    for state, pattern in state_patterns.items():
        if re.search(pattern, text, re.IGNORECASE):
            return state
    return "National / Unknown"
# extract data source names
def extract_datasource(url: str) -> str:
    domain = urlparse(url).netloc
    return domain.replace("www.", "")
# # --- 3. Target extractor ---
# remove stop words
STOPWORDS = set(stopwords.words("english"))
def clean_headline(text: str) -> str:
    # Step 1: remove trailing " - Source Name" or " – Source Name"
    text = re.sub(r"\s*[-–]\s+[A-Za-z0-9&.,'’ ]+$", "", text)
    # Step 2: remove stopwords
    words = text.lower().split()
    words = [w for w in words if w not in STOPWORDS]
    return " ".join(words)
######################################################################################################################################333
# compile patterns once (preserves insertion order = priority)
COMPILED_PATTERNS = [(canon, [re.compile(p, re.IGNORECASE) for p in pats])
                     for canon, pats in keywords.items()]

def extract_target_generic(text: str) -> str:
    t = clean_headline(text)
    m = re.search(r"\bban(?:s|ning|ned)?\s+(?:on\s+)?([A-Za-z0-9\- ]+)", t, re.IGNORECASE)
    if m:
        words = [w for w in m.group(1).strip().split() if w not in STOPWORDS]
        return " ".join(words[:3]) if words else None
    return None

def find_keyword_targets(headline: str, max_keywords: int = 3):
    """Return up to `max_keywords` canonical targets found in headline, by priority."""
    found = []
    for canon, patterns in COMPILED_PATTERNS:
        if any(p.search(headline) for p in patterns):
            found.append(canon)
            if len(found) >= max_keywords:
                break
    return found

def choose_target(headline: str) -> str | None:
    # 1) Keyword-first (robust to acronyms like CBD, brand names, etc.)
    kws = find_keyword_targets(headline, max_keywords=3)
    if kws:
        return ", ".join(kws)
    # 2) Fallback to regex phrase after "ban..."
    return extract_target_generic(headline)

In [7]:
# call scraping functions
df = pd.DataFrame(data=container)
df["state"] = df["headlines"].apply(extract_state)
df["datasource"] = df["article_links"].apply(extract_datasource)
df["target"] = df["headlines"].apply(choose_target)
df = df[['alert_date', 'state', 'target', 'headlines', 'datasource', 'article_links']].sort_values(by=['state', 'target']).drop_duplicates(subset=['article_links']).reset_index(drop=True)
df

,alert_date,state,target,headlines,datasource,article_links
0,"Mon, 15 Dec 2025 15:19:58 -0800",Alabama,DEI,"Professors, students appeal ruling on Alabama ...",wsfa.com,https://www.wsfa.com/2025/12/15/professors-stu...
1,"Thu, 25 Dec 2025 16:32:49 -0800",Alabama,america alabama brought,When Christmas Was Banned in America and How A...,obawebsite.com,https://www.obawebsite.com/when-christmas-was-...
2,"Mon, 17 Nov 2025 11:19:56 -0800",Alabama,cannabis,"Congress follows Alabama's lead, passes federa...",alreporter.com,https://www.alreporter.com/2025/11/17/congress...
3,"Fri, 14 Nov 2025 13:43:52 -0800",Alabama,cannabis,Shutdown bill could ban nearly all THC product...,al.com,https://www.al.com/news/2025/11/shutdown-bill-...
4,"Sat, 15 Nov 2025 15:58:37 -0800",Alabama,carbon storage project,Alabama lawmakers propose a bill to ban a carb...,wtvy.com,https://www.wtvy.com/2025/11/14/alabama-lawmak...
...,...,...,...,...,...,...
10744,"Thu, 09 Oct 2025 08:28:03 -0700",Wyoming,fossil fuels,Natrona County lifts open burning ban - Casper...,oilcity.news,https://oilcity.news/community/county-communit...
10745,"Sat, 27 Sep 2025 12:36:46 -0700",Wyoming,gender-affirming care,Plaintiffs launch final appeal to ban trans wo...,wyomingpublicmedia.org,https://www.wyomingpublicmedia.org/education/2...
10746,"Sat, 08 Nov 2025 09:59:14 -0800",Wyoming,None,Judge Upholds Wyoming's 'Sore Loser' Candidate...,cowboystatedaily.com,https://cowboystatedaily.com/2025/11/07/judge-...
10747,"Thu, 06 Nov 2025 15:15:11 -0800",Wyoming,None,Wyoming legislative committee backs felony bal...,county17.com,https://county17.com/2025/11/06/wyoming-legisl...


In [8]:
# df.to_excel('test_ban_file v4.xlsx', index=False)
df.info
print('total geographies:',len(df['state'].unique()), df.shape)
print('total geographies:',len(df[df['state'] != 'National / Unknown']['state'].unique()), df[df['state'] != 'National / Unknown'].shape)
df_filtered = df[df['state'] != 'National / Unknown']

detail_payload = (
    df_filtered[
        [
            "alert_date",
            "state",
            "target",
            "headlines",
            "datasource",
            "article_links",
        ]
    ]
    .sort_values(["alert_date", "state", "target"])
)
detail_payload

total geographies: 53 (10749, 6)
total geographies: 52 (3723, 6)


,alert_date,state,target,headlines,datasource,article_links
253,"Fri, 02 Jan 2026 05:41:07 -0800",California,"masks, police",California Is Banning Masks for Federal Agents...,independent.com,https://www.independent.com/2026/01/01/califor...
358,"Fri, 02 Jan 2026 05:41:07 -0800",California,plastic bags,California's tighter plastic bag ban takes eff...,actionnewsnow.com,https://www.actionnewsnow.com/news/butte-count...
1529,"Fri, 02 Jan 2026 05:41:07 -0800",Minnesota,sba loan programs,"Nearly 7,000 Minnesota borrowers banned from S...",kttc.com,https://www.kttc.com/2026/01/02/us-small-busin...
2296,"Fri, 02 Jan 2026 05:41:07 -0800",National,None,Federal employees file complaint against Trump...,weareiowa.com,https://www.weareiowa.com/article/nation-world...
10536,"Fri, 02 Jan 2026 05:41:07 -0800",Virginia,takes effect,West Virginia SNAP soda ban takes effect; loca...,wtrf.com,https://www.wtrf.com/news/west-virginia-snap-s...
...,...,...,...,...,...,...
9649,"Wed, 31 Dec 2025 16:36:48 -0800",New York,raspberry pi flipper,New York's Mayoral Inauguration bans Raspberry...,tomshardware.com,https://www.tomshardware.com/raspberry-pi/new-...
363,"Wed, 31 Dec 2025 18:17:00 -0800",California,plastic bags,California's 'Improved' Plastic Bag Ban Goes I...,plasticstoday.com,https://www.plasticstoday.com/legislation-regu...
568,"Wed, 31 Dec 2025 18:17:00 -0800",Colorado,effect new year,Firework bans in effect for New Year's Eve ami...,cpr.org,https://www.cpr.org/2025/12/31/firework-bans-n...
65,"Wed, 31 Dec 2025 23:21:38 -0800",Arkansas,burn ban,Arkansas counties under current burn bans | Wh...,thv11.com,https://www.thv11.com/article/news/local/arkan...


In [ ]:
# # --- Extract full headline from article ---
# def scrape_headline(url):
#     try:
#         r = requests.get(url, timeout=10, headers={"User-Agent":"Mozilla/5.0"})
#         r.raise_for_status()
#     except:
#         return None
#     soup = BeautifulSoup(r.text, "html.parser")
#     for sel in ['meta[property="og:title"]','meta[name="twitter:title"]','title']:
#         tag = soup.select_one(sel)
#         if tag:
#             return tag.get("content") if tag.has_attr("content") else tag.text
#     return None

# # --- Extract publish date from article ---
# def scrape_publish_date(url):
#     try:
#         r = requests.get(url, timeout=10, headers={"User-Agent":"Mozilla/5.0"})
#         r.raise_for_status()
#     except:
#         return None
#     soup = BeautifulSoup(r.text, "html.parser")
#     for sel in ['meta[property="article:published_time"]','meta[name="pubdate"]',
#                 'meta[name="date"]','time[datetime]']:
#         tag = soup.select_one(sel)
#         if tag:
#             return tag.get("content") if tag.has_attr("content") else tag.text
#     return None

In [ ]:
# # Full scraped headline
# df["headline_2"] = df["article_links"].apply(scrape_headline)

# # Published date
# df["article_publish_date"] = df["article_links"].apply(scrape_publish_date)

In [9]:
df[['alert_date', 'state', 'target', 'headlines', 'datasource', 'article_links']]# .to_excel('test_ban_file 8.30.2025 v2.xlsx', index=False)

,alert_date,state,target,headlines,datasource,article_links
0,"Mon, 15 Dec 2025 15:19:58 -0800",Alabama,DEI,"Professors, students appeal ruling on Alabama ...",wsfa.com,https://www.wsfa.com/2025/12/15/professors-stu...
1,"Thu, 25 Dec 2025 16:32:49 -0800",Alabama,america alabama brought,When Christmas Was Banned in America and How A...,obawebsite.com,https://www.obawebsite.com/when-christmas-was-...
2,"Mon, 17 Nov 2025 11:19:56 -0800",Alabama,cannabis,"Congress follows Alabama's lead, passes federa...",alreporter.com,https://www.alreporter.com/2025/11/17/congress...
3,"Fri, 14 Nov 2025 13:43:52 -0800",Alabama,cannabis,Shutdown bill could ban nearly all THC product...,al.com,https://www.al.com/news/2025/11/shutdown-bill-...
4,"Sat, 15 Nov 2025 15:58:37 -0800",Alabama,carbon storage project,Alabama lawmakers propose a bill to ban a carb...,wtvy.com,https://www.wtvy.com/2025/11/14/alabama-lawmak...
...,...,...,...,...,...,...
10744,"Thu, 09 Oct 2025 08:28:03 -0700",Wyoming,fossil fuels,Natrona County lifts open burning ban - Casper...,oilcity.news,https://oilcity.news/community/county-communit...
10745,"Sat, 27 Sep 2025 12:36:46 -0700",Wyoming,gender-affirming care,Plaintiffs launch final appeal to ban trans wo...,wyomingpublicmedia.org,https://www.wyomingpublicmedia.org/education/2...
10746,"Sat, 08 Nov 2025 09:59:14 -0800",Wyoming,None,Judge Upholds Wyoming's 'Sore Loser' Candidate...,cowboystatedaily.com,https://cowboystatedaily.com/2025/11/07/judge-...
10747,"Thu, 06 Nov 2025 15:15:11 -0800",Wyoming,None,Wyoming legislative committee backs felony bal...,county17.com,https://county17.com/2025/11/06/wyoming-legisl...


In [ ]:
# NOTES
# drop duplicates from dataframe based on url, can be done first thing [done]
# reorganize pipeline to scrape headlines and publish dates from urls, fill in blanks with scraping from google alert, run regex on states and targets on aggregated column
# --- see if I can make my pipeline flexible enough to easily toggle on/off url scraping since that currently has a ~15 min runtime
# figure out what the fuck is going on with Indiana [done]

# look into graphics and summary views
# how to make a dashboard I can host on AWS?

In [10]:
# Group and clean
state_counts = df[~df['state'].str.contains('National')].groupby("state").agg(
    count_of_news_articles=("target", "count"),
    targets=("target", lambda x: ", ".join(sorted({t for t in x if t})))
).reset_index()

# Map state names -> abbreviations
state_counts["state_abbrev"] = state_counts["state"].apply(
    lambda x: us.states.lookup(x).abbr if us.states.lookup(x) else None
)

# Choropleth
fig = px.choropleth(
    state_counts,
    locations="state_abbrev",
    locationmode="USA-states",
    color="count_of_news_articles",
    scope="usa",
    color_continuous_scale="Reds",
    title="News Articles About Bans by State",
    hover_data={"targets": True,
                "count_of_news_articles": True, 
                "state": True},
    height=600,
    width=1200
)

fig.update_layout(
    title_x=0.5
)

In [11]:
print('earliest date:',pd.to_datetime(df['alert_date'],utc=True).max())
print('latest date:',pd.to_datetime(df['alert_date'], utc=True).min())
# Step 1: Aggregate unique datasources and states per target, remove duplicates
hover_df = df.groupby("target").agg(
    count_of_news_articles=("target", "count"),
    datasources=("datasource", lambda x: ", ".join(sorted(set(x)))),
    states=("state", lambda x: ", ".join(sorted(set(x))))
).reset_index()

top_n = 15
# Step 2: Keep top 15 by count
hover_df = hover_df.sort_values("count_of_news_articles", ascending=False).head(top_n)

# Step 3: Plot histogram with hover data
fig = px.bar(
    hover_df,
    title=f"{top_n} Most Common Topics of News Articles Relating to Bans",
    x="target",
    y="count_of_news_articles",
    hover_data={
        "datasources": True,
        "states": True,
        "count_of_news_articles": True,
        "target": False
    },
    text="count_of_news_articles",
    height=600,
    width=1200
)
fig.update_layout(
    title_x=0.5
)

earliest date: 2026-01-18 22:11:41+00:00
latest date: 2025-08-30 23:21:50+00:00


In [12]:
# Group and clean
state_counts = df.groupby("alert_date").agg(
    count_of_news_articles=("target", "count"),
    targets=("target", lambda x: ", ".join(sorted({t for t in x if t})))
).reset_index()

state_counts['date']=pd.to_datetime(state_counts['alert_date'])

fig = px.line(
    state_counts.sort_values(by="date", ascending=True),
    x="date",
    y="count_of_news_articles",
    title="Timeline of News Articles About Bans In The US",
    height=600,
    width=1400
)

fig.update_layout(
    title_x=0.5
)

C:\Users\makri\AppData\Local\Temp\ipykernel_17732\1138019820.py:7: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



In [ ]:
# df.to_excel('test 9.21.xlsx', index=False)